In [1]:
import copy
import pickle
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
G_train = nx.read_edgelist('t1-train.txt', create_using=nx.DiGraph(), nodetype=int, edgetype=int)
unG_train = nx.read_edgelist('t1-train.txt', create_using=nx.Graph(), nodetype=int, edgetype=int)
print(len(G_train.edges), 'edges')
print(len(G_train.nodes), 'nodes')

234983 edges
24158 nodes


In [3]:
G_test_seen = nx.read_edgelist('t1-test-seen.txt', create_using=nx.DiGraph(), nodetype=int, edgetype=int)
print(len(G_test_seen.edges), 'edges')
print(len(G_test_seen.nodes), 'nodes')

50806 edges
16325 nodes


In [4]:
G_test = nx.read_edgelist('t1-test.txt', create_using=nx.DiGraph(), nodetype=int, edgetype=int)
print(len(G_test.edges), 'edges')
print(len(G_test.nodes), 'nodes')

test_x = []
for fromId, toId in np.loadtxt('t1-test.txt'):
    test_x.append((fromId,  toId))
test_x = np.array(test_x).astype('int64')

88074 edges
18726 nodes


In [5]:
GG = nx.DiGraph()
GG.add_edges_from(list(G_test_seen.edges))
GG.add_edges_from(list(G_train.edges()))
unGG = nx.Graph()
unGG.add_edges_from(list(GG.edges()))

In [6]:
data2 = []
label2 = []
size = []

# t1-train
print('t1-train')
for e, node in enumerate(GG.nodes):
    
    pos_nodes = set(GG.successors(node))
    
    neg_nodes = set()
    cand = [node]
    for _ in range(4): # 3步往前
        new = set()
        for i in cand:
            tmp = set(GG.successors(i))
            new = new | tmp
        new = new - neg_nodes - set([node])
        neg_nodes = neg_nodes | new
        cand = list(new)
    
    q = set(G_test.successors(node)) if node in G_test else set()
    neg_nodes = neg_nodes - pos_nodes - set([node]) - q
    size.append(len(neg_nodes))
    neg_samples = np.random.choice(list(neg_nodes), int(len(pos_nodes) * 1)) if neg_nodes else []


    for pos in pos_nodes:
        data2.append((node, pos))
        label2.append(1)

    for neg in neg_samples:
        data2.append((node, neg))
        label2.append(0)

    print(np.round(e/len(GG), 2), end='\r')
print(1.00)

print(np.mean(size))
print(len(data2), len(label2))

t1-train
1.09
866.5618325283995
568095 568095


In [7]:
%store data2
%store label2

Stored 'data2' (list)
Stored 'label2' (list)


In [18]:
size = []

# t1-train
print('t1-test-seen')
for e, node in enumerate(G_test_seen.nodes):
    
    pos_nodes = set(GG.successors(node))
    
    neg_nodes = set(nx.all_neighbors(unGG, node))# start from neighbor
    cand = neg_nodes
    for _ in range(3): # 3步往前
        new = set()
        for i in cand:
            tmp = set(GG.successors(i))
            new = new | tmp
            neg_nodes = neg_nodes | tmp
        cand = new
        
    q = set(G_test.successors(node)) if node in G_test else set()
    neg_nodes = neg_nodes - pos_nodes - set([node]) - q
    size.append(len(neg_nodes))
    neg_samples = np.random.choice(list(neg_nodes), int(len(pos_nodes) * 1)) if neg_nodes else []


    for pos in pos_nodes:
        data.append((node, pos))
        label.append(1)

    for neg in neg_samples:
        data.append((node, neg))
        label.append(0)

    print(np.round(e/len(G_test_seen.nodes), 2), end='\r')
print(1.00)

print(np.mean(size))
print(len(data), len(label))

t1-train
1.08
2936.2041041347625
859016 859016


In [20]:
859016 - 478709

380307

In [19]:
%store data
%store label

Stored 'data' (list)
Stored 'label' (list)


In [22]:
np.savetxt("train_x.csv", train_x, delimiter=",")
np.savetxt("train_y.csv", train_y, delimiter=",")
np.savetxt("val_x.csv", val_x, delimiter=",")
np.savetxt("val_y.csv", val_y, delimiter=",")

%store train_x
%store train_y
%store val_x
%store val_y

Stored 'train_x' (ndarray)
Stored 'train_y' (ndarray)
Stored 'val_x' (ndarray)
Stored 'val_y' (ndarray)


In [4]:
size = []

In [11]:
def gen_test_data(nxGraph):
    print('gen sample')
    data = []
    label = []
    
    
    for e, node in enumerate(nxGraph):
        pos_nodes = set(nxGraph.successors(node))
        q = set(G_test.successors(node)) if node in G_test else set()
        
        neg_nodes = set(nx.descendants(nxGraph, node)) - pos_nodes - set([node]) - q
        size.append(len(neg_nodes))
        neg_samples = np.random.choice(list(neg_nodes), int(len(pos_nodes) * 1)) if neg_nodes else []
        
        for pos in pos_nodes:
            data.append((node, pos))
            label.append(1)
            
        for neg in neg_samples:
            data.append((node, neg))
            label.append(0)
        
        print(np.round(e/len(nxGraph), 2), end='\r')
    print(1.00)
    
    return np.array(data).astype('int64'), np.array(label).astype('float32')

In [12]:
all_x, all_y = gen_test_data(GG)

gen sample
1.09


In [13]:
np.mean(size)

5240.794299707503

In [28]:
%store all_x
%store all_y

Stored 'all_x' (ndarray)
Stored 'all_y' (ndarray)


In [23]:
test_seen_x = []

for fromId, toId in G_test_seen.edges:
    if fromId not in G_train or toId not in G_train:
        continue
    test_seen_x.append((fromId,  toId))

test_seen_x = np.array(test_seen_x).astype('float32')
test_seen_y = np.ones(test_seen_x.shape[0])

In [24]:
%store G
%store unG
%store G_train
%store unG_train
%store G_val
%store G_test_seen
%store GG
%store unGG
%store train_x
%store train_y
%store val_x
%store val_y
%store test_seen_x
%store test_seen_y
%store test_x
%store all_x
%store all_y

Stored 'G' (DiGraph)
Stored 'unG' (Graph)
Stored 'G_train' (DiGraph)
Stored 'unG_train' (Graph)
Stored 'G_val' (DiGraph)
Stored 'G_test_seen' (DiGraph)
Stored 'GG' (DiGraph)
Stored 'unGG' (Graph)
Stored 'train_x' (ndarray)
Stored 'train_y' (ndarray)
Stored 'val_x' (ndarray)
Stored 'val_y' (ndarray)
Stored 'test_seen_x' (ndarray)
Stored 'test_seen_y' (ndarray)
Stored 'test_x' (ndarray)
Stored 'all_x' (ndarray)
Stored 'all_y' (ndarray)


In [25]:
tmp = set()
for i, j in zip(all_x, all_y):
    if j == 1:
        tmp.add(tuple(i))
len(set(G_test_seen.edges) - set(tmp))

0